## Load Modules +Load Data 

In [3]:
# Load modules
from dash import Dash, dcc, html, Input, Output,callback
from jupyter_dash import JupyterDash
import dash_bootstrap_components as dbc
import plotly.express as px

import altair as alt
import numpy as np
import pandas as pd

murder = pd.read_csv("database.csv")

#remove space to conduct query
murder.columns =['Record_ID', 'Agency_Code', 'Agency_Name', 'Agency_Type', 'City',
       'State', 'Year', 'Month', 'Incident', 'Crime_Type', 'Crime_Solved',
       'Victim_Sex', 'Victim_Age', 'Victim_Race', 'Victim_Ethnicity',
       'Perpetrator_Sex', 'Perpetrator_Age', 'Perpetrator_Race',
       'Perpetrator_Ethnicity', 'Relationship', 'Weapon', 'Victim_Count',
       'Perpetrator_Count', 'Record_Source']

/var/folders/21/p7sl2l2j0s37kbd62tb3_qt00000gn/T/ipykernel_883/3418500140.py:11: DtypeWarning:

Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.



## Build Dashboard 

In [ ]:

# Instantiate the app

#Jupyter dash
app = JupyterDash(external_stylesheets=[dbc.themes.BOOTSTRAP])


server = app.server


#### Set Layout
app.layout = dbc.Container([
    dbc.Tabs([
        dbc.Tab([
            html.H1("Put your code in between [ ] below."),            
                ],
        label = "Tab One"
               ),
        
        dbc.Tab([
            
            html.H1("Dashboard of Murder Cases by Months"),
    
            html.P("Please select features to do filtering on"),
            
            dbc.Row([
                # Dropdown 
                dbc.Col(
                    [
                        html.Br(),
                       
                        html.Label("Crime Type: "),
                        dcc.Dropdown(
                            id = "input_crime_type",
                            value= "Murder or Manslaughter",
                            multi = False,
                            clearable = False,
                            style = {"font-size": "smaller"},
                            options= [
                                        {"label": crime_type, "value": crime_type} for crime_type in list(murder["Crime_Type"].dropna().unique())
                                     ]
                                    ),
                        
                        html.Br(),
                        
                        html.Label("State: "),
                        dcc.Dropdown(
                            id = "input_state",
                            value = "Alaska",
                            multi = False,
                            clearable = True,
                            style = {"font-size": "smaller"},
                            options = [
                                        {"label": state, "value": state} for state in list(murder["State"].dropna().unique())
                                      ]
                                    ),
                        
                        html.Br(),
                        
                        html.Label("Weapon Type: "),
                        dcc.Dropdown(
                            id = "input_weapon",
                            value = "Blunt Object",
                            multi = False,
                            clearable = True,
                            style = {"font-size": "smaller"},
                            options = [
                                        {"label": weapon, "value": weapon} for weapon in list(murder["Weapon"].dropna().unique())
                                      ]
                                    ),
                        
                        html.Br(),
                       
                        html.Label("Relationship with Victim: "),
                        dcc.Dropdown(
                            id = "input_relationship",
                            value = "Acquaintance",
                            multi = False,
                            clearable = True,
                            style = {"font-size": "smaller"},
                            options = [
                                        {"label": relationship, "value": relationship} for relationship in list(murder["Relationship"].dropna().unique())
                                      ],
                        )
                    ],
                    md=4,
                    style={"border": "1px solid #d3d3d3",
                           "border-radius": "10px",
                           "background-color": "rgba(220, 220, 220, 0.5)"
                          }
                ),                                           
            
            
                #Major Rose Graph
                dbc.Col(
                    dcc.Graph(id = "rose_plot"
                             )      
                       )

            ])            
            ],
        label = "Tab Two"
        )
    
])
])



#### Callback 
@app.callback(
    Output("rose_plot", "figure"),
    Input("input_crime_type", "value"),
    Input("input_state", "value"),
    Input("input_weapon", "value"),
    Input("input_relationship", "value")
             )


def plot_rose(input_crime_type,input_state,input_weapon,input_relationship):
    #filter 
    murder_selected = murder.loc[(murder["Crime_Type"] == input_crime_type) & (murder["State"] == input_state) 
                                 & (murder["Weapon"] == input_weapon) & (murder["Relationship"] == input_relationship)]
       
    
    # Produce needed summary df 
    case_by_month = murder_selected.Record_ID.groupby(murder.Month).count().reset_index(level=0)
    
    chart = px.bar_polar(case_by_month, r = "Record_ID", theta = "Month")
    
    chart.update_layout(transition_duration = 500)
    return chart



#https://plotly.com/python/wind-rose-charts/
#https://altair-viz.github.io/gallery/radial_chart.html



if __name__ == '__main__':
    app.server.run(port=8000, host='127.0.0.1')






 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8000/ (Press CTRL+C to quit)
